In [ ]:
import sys
sys.path.append('../../')

import estraces
import scared
import numpy as np
from kresca.plot import CurvePlot
from kresca.reverse import LeakageAssessment
from kresca.attack import NonProfileAttack

## Import Traces

In [ ]:
ths = estraces.read_ths_from_ets_file('../../Traces/ETS/Telkom_30k_sync2.ets')

In [ ]:
print(ths)

## Plot Traces

In [ ]:
#put inline=True if you want to display on notebook
CurvePlot(ths.samples[:5], inline=True) 

## Milenage Attack

### Milenage First SubBytes

In [ ]:
@scared.attack_selection_function
def milenage_first_subbytes(plaintext, guesses):
    res = np.empty((plaintext.shape[0], len(guesses), plaintext.shape[1]), dtype='uint8')
    for i, guess in enumerate(guesses):
        res[:, i, :] = scared.aes.SBOX[np.bitwise_xor(plaintext, guess)]
    return res

In [ ]:
Milenage_selection_functions = {
    'Milenage First Sub Bytes': milenage_first_subbytes,
}

In [ ]:
attack = NonProfileAttack(
    ths=ths,
    selection_functions=Milenage_selection_functions
)

In [ ]:
attack.run()

In [ ]:
attack.report()

In [ ]:
attack.show_result()

### Milenage Second Subbytes

In [ ]:
key_xor_opc = np.array([0x71, 0x6d, 0xe2, 0xfb, 0xa0, 0x8a, 0xdf, 0xd, 0xf2, 0x26, 0x12, 0xe1, 0xe5, 0xc4, 0x3f, 0x3], dtype='uint8')

@scared.attack_selection_function
def milenage_second_subbytes(plaintext, guesses):
    res = np.empty((plaintext.shape[0], len(guesses), plaintext.shape[1]), dtype='uint8')
    output_aes_round_1 = scared.aes.encrypt(plaintext, key_xor_opc, at_round=1, after_step=scared.aes.Steps.MIX_COLUMNS)
    for i, guess in enumerate(guesses):
        res[:, i, :] = scared.aes.SBOX[np.bitwise_xor(output_aes_round_1, guess)]
    return res

In [ ]:
Milenage_selection_functions = {
    'Milenage Second Sub Bytes': milenage_second_subbytes,
}

In [ ]:
attack = NonProfileAttack(
    ths=ths,
    selection_functions=Milenage_selection_functions
)

In [ ]:
attack.run()

In [ ]:
attack.report()

In [ ]:
attack.show_result()

In [ ]:
aes_key_2 = np.array([0xa, 0x1, 0x7, 0xc4, 0x18, 0x43, 0xab, 0x6e, 0x6, 0x7b, 0x4d, 0xd, 0xfb, 0x3, 0xc6, 0x18], dtype='uint8')

## Recover Master Key

In [ ]:
def array_to_hexstring(data):
    return "".join([f"{i:2X}" for i in data])

In [ ]:
master_key = scared.aes.inv_key_schedule(aes_key_2, round_in=1)[0][0]
opc = np.bitwise_xor(master_key, key_xor_opc)

print(f"Key : {array_to_hexstring(master_key)}")
print(f"Opc : {array_to_hexstring(opc)}")